# Time series tests

In [17]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from statsmodels.tsa.ar_model import AR,ARResults
import warnings
warnings.filterwarnings('ignore')

In [10]:
EuStocks = pd.read_csv("EuIndices (1).csv")

In [32]:
EuStocks_DF = pd.DataFrame(data = EuStocks.values, columns = ['0', 'DAX', 'SMI', 'CAC', 'FTSE'],
                           index = pd.DatetimeIndex(start = '1991-01-01', periods = 1860, freq = 'B'))
df = EuStocks_DF.drop(columns = ['0'])
df.head()

,DAX,SMI,CAC,FTSE
1991-01-01,1628.75,1678.1,1772.8,2443.6
1991-01-02,1613.63,1688.5,1750.5,2460.2
1991-01-03,1606.51,1678.6,1718.0,2448.2
1991-01-04,1621.04,1684.1,1708.1,2470.4
1991-01-07,1618.16,1686.6,1723.1,2484.7


# Augmented Dickey-Fuller Test (Stationary Test)

In [30]:
from statsmodels.tsa.stattools import adfuller

In [33]:
def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): 
        return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")
          
# ADF Test on each column
for name, column in df.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "DAX" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = 2.2958
 No. Lags Chosen       = 24
 Critical value 1%     = -3.434
 Critical value 5%     = -2.863
 Critical value 10%    = -2.568
 => P-Value = 0.999. Weak evidence to reject the Null Hypothesis.
 => Series is Non-Stationary.


    Augmented Dickey-Fuller Test on "SMI" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = 2.235
 No. Lags Chosen       = 22
 Critical value 1%     = -3.434
 Critical value 5%     = -2.863
 Critical value 10%    = -2.568
 => P-Value = 0.9989. Weak evidence to reject the Null Hypothesis.
 => Series is Non-Stationary.


    Augmented Dickey-Fuller Test on "CAC" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. N

#### Entire dataset is non stationary

# Granger’s Causality Test

In [39]:
X_train=df.iloc[:1302]
X_test=df.iloc[1302:]

In [41]:
from statsmodels.tsa.stattools import grangercausalitytests
maxlag=12
test = 'ssr_chi2test'
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    X_train = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in X_train.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            X_train.loc[r, c] = min_p_value
    X_train.columns = [var + '_x' for var in variables]
    X_train.index = [var + '_y' for var in variables]
    return X_train

grangers_causation_matrix(X_train, variables = X_train.columns)

,DAX_x,SMI_x,CAC_x,FTSE_x
DAX_y,1.0000,0.0001,0.6516,0.0046
SMI_y,0.1210,1.0000,0.0810,0.0055
CAC_y,0.4228,0.0109,1.0000,0.2846
FTSE_y,0.0216,0.0006,0.0147,1.0000


#### None of the variables are causing effect on other on either direction. This is because p value >0.05 for some variables. If all are <0.05 then there is a causality effect between those variables.

# Durbin Watson Statistic

#### 2 is no autocorrelation.
#### 0 to <2 is positive autocorrelation (common in time series data).
#### 2 to 4 is negative autocorrelation (less common in time series data).

####  If there is any correlation left in the residuals, then, there is some pattern in the time series that is still left to be explained by the model. In that case, the typical course of action is to either increase the order of the model or induce more predictors into the system or look for a different algorithm to model the time series. So, checking for serial correlation is to ensure that the model is sufficiently able to explain the variances and patterns in the time series.

In [ ]:
from statsmodels.stats.stattools import durbin_watson
out = durbin_watson(model_fitted.resid)

for col, val in zip(X_train.columns, out):
    print((col), ':', round(val, 2))